In [1]:
import pandas as pd

In [2]:
# Load the Excel file into a DataFrame
poprad_df = pd.read_excel("resampled_data_3.xlsx")


In [3]:
# Iterate over the columns of the DataFrame
for col in poprad_df.columns:
    # Check if the column name contains the string "distance"
    if "distance" in col:
        # Extract the sensor number from the column name
        sensor = col.split(".")[0]

        # Create a new column for the difference if it doesn't already exist
        if f"{sensor}.difference" not in poprad_df.columns:
            poprad_df[f"{sensor}.difference"] = poprad_df[f"{sensor}.distance_2"] - \
                poprad_df[f"{sensor}.distance_1"]

In [4]:
poprad_df.columns

Index(['datetime', '1.distance_1', '1.distance_2', '2.distance_1',
       '2.distance_2', '2.temperature_external_1', '2.temperature_external_2',
       '3.distance_1', '3.distance_2', '3.temperature_external_1',
       '3.temperature_external_2', '4.distance_1', '4.distance_2',
       '4.temperature_external_1', '4.temperature_external_2', '6.distance_1',
       '6.distance_2', '6.temperature_external_1', '6.temperature_external_2',
       '7.distance_1', '7.distance_2', '9.distance_1', '9.distance_2',
       '9.temperature_external_1', '9.temperature_external_2', '10.distance_1',
       '10.distance_2', '10.temperature_external_2', '11.distance_1',
       '11.distance_2', '12.distance_1', '12.distance_2', '13.distance_1',
       '13.distance_2', '1.difference', '2.difference', '3.difference',
       '4.difference', '6.difference', '7.difference', '9.difference',
       '10.difference', '11.difference', '12.difference', '13.difference'],
      dtype='object')

In [5]:
import pandas as pd
import plotly.graph_objects as go


def plot_sensor_data(poprad_df, color='#b58900', sensor=13):
    plot_df = poprad_df.copy()
    plot_df['datetime'] = pd.to_datetime(plot_df['datetime'])
    sensor = str(sensor)

    # Extract the two distances for the sensor
    plot_df['distance_1'] = plot_df[f'{sensor}.distance_1']
    plot_df['distance_2'] = plot_df[f'{sensor}.distance_2']
    plot_df['difference'] = plot_df[f'{sensor}.difference']

    # Grouping by year_month
    grouped_df = plot_df.groupby(plot_df['datetime'].dt.to_period('M'))

    for year_month, group in grouped_df:
        # Create a separate plot for each month
        fig = go.Figure()

        # Plot distance_1 and distance_2 as lines
        fig.add_trace(go.Scatter(
            x=group['datetime'], y=group['distance_1'], mode='lines', name=f'Snímač {sensor}',
            line_shape='spline', line_color=color))
        fig.add_trace(go.Scatter(
            x=group['datetime'], y=group['distance_2'], mode='lines', name='Referenčný',
            line_shape='spline', line_dash='dash', line_color=color))

        # Plot difference with its own secondary y-axis
        fig.add_trace(go.Scatter(
            x=group['datetime'], y=group['difference'], mode='lines', name='Rozdiel',
            line_shape='spline', line_color='#0A0702', line_width=3,
            yaxis='y2'))

        # Compute minimum and maximum difference
        min_difference = group['difference'].min()
        max_difference = group['difference'].max()

        # Add annotations for minimum and maximum difference to the bottom right
        fig.add_annotation(
            x=0.95, y=0.05, xref="paper", yref="paper",
            text=f"Min: {min_difference:.3f} mm",
            showarrow=False
        )
        fig.add_annotation(
            x=0.95, y=0.02, xref="paper", yref="paper",
            text=f"Max: {max_difference:.3f} mm",
            showarrow=False
        )

        # Create axes objects
        fig.update_layout(
            xaxis_title='Dátum',
            yaxis=dict(title='Vzdialenosť [mm]', side='left',
                       showgrid=False, range=[-0.15, 0.15]),
            yaxis2=dict(
                title='Rozdiel [mm]', overlaying='y', side='right', showgrid=False, range=[-0.05, 0.05]),
            title=f'Senzor {sensor} - {year_month}',
            legend=dict(orientation='h', yanchor='bottom',
                        y=1.02, xanchor='right', x=1),
            plot_bgcolor='#EEEEEE',
            width=850, height=450,
            margin=dict(l=20, r=25, t=50, b=40),
            font_family="Montserrat"
        )

        # Show the plot
        fig.show()

In [6]:
import pandas as pd
import plotly.graph_objects as go


def plot_sensor_data_monthly(poprad_df, color='#b58900', sensor=13):
    plot_df = poprad_df.copy()
    plot_df['datetime'] = pd.to_datetime(plot_df['datetime'])
    sensor = str(sensor)

    # Extract the two distances for the sensor
    plot_df['distance_1'] = plot_df[f'{sensor}.distance_1']
    plot_df['distance_2'] = plot_df[f'{sensor}.distance_2']
    plot_df['difference'] = plot_df[f'{sensor}.difference']

    # Create the plot
    fig = go.Figure()

    # Plot distance_1 and distance_2 as lines
    fig.add_trace(go.Scatter(
        x=plot_df['datetime'], y=plot_df['distance_1'], mode='lines', name=f'Snímač {sensor}',
        line_shape='spline', line_color=color))
    fig.add_trace(go.Scatter(
        x=plot_df['datetime'], y=plot_df['distance_2'], mode='lines', name='Referenčný',
        line_shape='spline', line_dash='dash', line_color=color))

    # Plot difference with its own secondary y-axis
    fig.add_trace(go.Scatter(
        x=plot_df['datetime'], y=plot_df['difference'], mode='lines', name='Rozdiel',
        line_shape='spline', line_color='#0A0702', line_width=3,
        yaxis='y2'))

    # Compute minimum and maximum difference
    min_difference = plot_df['difference'].min()
    max_difference = plot_df['difference'].max()

    # Add annotations for minimum and maximum difference to the bottom right
    fig.add_annotation(
        x=0.95, y=0.05, xref="paper", yref="paper",
        text=f"Min: {min_difference:.3f} mm",
        showarrow=False
    )
    fig.add_annotation(
        x=0.95, y=0.02, xref="paper", yref="paper",
        text=f"Max: {max_difference:.3f} mm",
        showarrow=False
    )

    # Create axes objects
    fig.update_layout(
        xaxis_title='Dátum',
        yaxis=dict(title='Vzdialenosť [mm]', side='left',
                   showgrid=False, range=[-0.15, 0.15]),
        yaxis2=dict(
            title='Rozdiel [mm]', overlaying='y', side='right', showgrid=False, range=[-0.05, 0.05]),
        title=f'Senzor {sensor}',
        legend=dict(orientation='h', yanchor='bottom',
                    y=1.02, xanchor='right', x=1),
        plot_bgcolor='#EEEEEE',
        width=850, height=450,
        margin=dict(l=20, r=25, t=50, b=40),
        font_family="Montserrat"
    )

    # Show the plot
    fig.show()

In [7]:
dictionary = {
    1: "#f77189",
    2: "#e18632",
    3: "#b59a32",
    4: "#8ba731",
    6: "#32b258",
    7: "#35ae95",
    9: "#37abb2",
    10: "#39a7d6",
    11: "#8f93f4",
    12: "#db70f4",
    13: "#f667c6"
    }

In [8]:
# Use this to plot and save plots
for sensor, color in dictionary.items():
    plot_sensor_data_monthly(poprad_df, color=color, sensor=sensor)

In [9]:
for sensor, color in dictionary.items():
    plot_sensor_data(poprad_df, color=color, sensor=sensor)

In [9]:
import pandas as pd
import plotly.graph_objects as go


def plot_sensor_data(poprad_df, color='#b58900', sensor=13):
    plot_df = poprad_df.copy()
    plot_df['datetime'] = pd.to_datetime(plot_df['datetime'])
    sensor = str(sensor)

    # Extract the two distances for the sensor
    plot_df['distance_1'] = plot_df[f'{sensor}.distance_1']
    plot_df['distance_2'] = plot_df[f'{sensor}.distance_2']
    plot_df['difference'] = plot_df[f'{sensor}.difference']

    # Grouping by year_month
    grouped_df = plot_df.groupby(plot_df['datetime'].dt.to_period('M'))

    for year_month, group in grouped_df:
        # Create a separate plot for each month
        fig = go.Figure()

        # Plot distance_1 and distance_2 as lines
        fig.add_trace(go.Scatter(
            x=group['datetime'], y=group['distance_1'], mode='lines', name=f'Snímač {sensor}',
            line_shape='spline', line_color=color))
        fig.add_trace(go.Scatter(
            x=group['datetime'], y=group['distance_2'], mode='lines', name='Referenčný',
            line_shape='spline', line_dash='dash', line_color=color))

        # Plot difference with its own secondary y-axis
        fig.add_trace(go.Scatter(
            x=group['datetime'], y=group['difference'], mode='lines', name='Rozdiel',
            line_shape='spline', line_color='#0A0702', line_width=3,
            yaxis='y2'))

        # Compute minimum and maximum difference
        min_difference = group['difference'].min()
        max_difference = group['difference'].max()

        # Add annotations for minimum and maximum difference to the bottom right
        fig.add_annotation(
            x=0.95, y=0.05, xref="paper", yref="paper",
            text=f"Min: {min_difference:.3f} mm",
            showarrow=False
        )
        fig.add_annotation(
            x=0.95, y=0.02, xref="paper", yref="paper",
            text=f"Max: {max_difference:.3f} mm",
            showarrow=False
        )

        # Create axes objects
        fig.update_layout(
            xaxis_title='Dátum',
            yaxis=dict(title='Vzdialenosť [mm]', side='left',
                       showgrid=False, range=[-0.25, 0.25]),
            yaxis2=dict(
                title='Rozdiel [mm]', overlaying='y', side='right', showgrid=False, range=[-0.05, 0.05]),
            title=f'Senzor {sensor} - {year_month}',
            legend=dict(orientation='h', yanchor='bottom',
                        y=1.02, xanchor='right', x=1),
            plot_bgcolor='#EEEEEE',
            width=850, height=450,
            margin=dict(l=20, r=25, t=50, b=40),
            font_family="Montserrat"
        )

        # Save the plot as a PNG file
        filename = f'sensor_{sensor}_{year_month}.png'
        fig.write_image(filename, width=850, height=450)

In [10]:
# Use this to plot and save plots
for sensor, color in dictionary.items():
    plot_sensor_data(poprad_df, color=color, sensor=sensor)

In [13]:
import pandas as pd
import plotly.graph_objects as go


def plot_sensor_data_monthly(poprad_df, color='#b58900', sensor=13):
    plot_df = poprad_df.copy()
    plot_df['datetime'] = pd.to_datetime(plot_df['datetime'])
    sensor = str(sensor)

    # Extract the two distances for the sensor
    plot_df['distance_1'] = plot_df[f'{sensor}.distance_1']
    plot_df['distance_2'] = plot_df[f'{sensor}.distance_2']
    plot_df['difference'] = plot_df[f'{sensor}.difference']

    # Create the plot
    fig = go.Figure()

    # Plot distance_1 and distance_2 as lines
    fig.add_trace(go.Scatter(
        x=plot_df['datetime'], y=plot_df['distance_1'], mode='lines', name=f'Snímač {sensor}',
        line_shape='spline', line_color=color))
    fig.add_trace(go.Scatter(
        x=plot_df['datetime'], y=plot_df['distance_2'], mode='lines', name='Referenčný',
        line_shape='spline', line_dash='dash', line_color=color))

    # Plot difference with its own secondary y-axis
    fig.add_trace(go.Scatter(
        x=plot_df['datetime'], y=plot_df['difference'], mode='lines', name='Rozdiel',
        line_shape='spline', line_color='#0A0702', line_width=3,
        yaxis='y2'))

    # Compute minimum and maximum difference
    min_difference = plot_df['difference'].min()
    max_difference = plot_df['difference'].max()

    # Add annotations for minimum and maximum difference to the bottom right
    fig.add_annotation(
        x=0.95, y=0.05, xref="paper", yref="paper",
        text=f"Min: {min_difference:.3f} mm",
        showarrow=False
    )
    fig.add_annotation(
        x=0.95, y=0.02, xref="paper", yref="paper",
        text=f"Max: {max_difference:.3f} mm",
        showarrow=False
    )

    # Create axes objects
    fig.update_layout(
        xaxis_title='Dátum',
        yaxis=dict(title='Vzdialenosť [mm]', side='left',
                   showgrid=False, range=[-0.15, 0.15]),
        yaxis2=dict(
            title='Rozdiel [mm]', overlaying='y', side='right', showgrid=False, range=[-0.05, 0.05]),
        title=f'Senzor {sensor}',
        legend=dict(orientation='h', yanchor='bottom',
                    y=1.02, xanchor='right', x=1),
        plot_bgcolor='#EEEEEE',
        width=850, height=450,
        margin=dict(l=20, r=25, t=50, b=40),
        font_family="Montserrat"
    )

    # Show the plot
    fig.show()

In [15]:
# Use this to plot and save plots
for sensor, color in dictionary.items():
    plot_sensor_data_monthly(poprad_df, color=color, sensor=sensor)

NameError: name 'plot_sensor_data_monthly' is not defined